In [1]:
import torch
print(torch.cuda.is_available())
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import torchvision
from torchvision import transforms
import pandas as pd
import numpy as np
from PIL import Image
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tqdm import tqdm

True


In [2]:
print(torch.cuda.get_device_name(0))

# --- Yapılandırma Parametreleri ---

# Veri Yolları
DATA_DIR = '../data/fashion-dataset/'
CSV_FILE = os.path.join(DATA_DIR, 'styles.csv')
IMAGES_DIR = os.path.join(DATA_DIR, 'images')

# Model Parametreleri
NUM_EPOCHS = 10
BATCH_SIZE = 64
LEARNING_RATE = 0.001
NUM_WORKERS = 2 # Veri yükleme hızını artırır

# Cihaz (GPU varsa kullanır)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Kullanılan Cihaz: {DEVICE}")

NVIDIA GeForce RTX 3070
Kullanılan Cihaz: cuda


In [3]:
df = pd.read_csv(CSV_FILE, on_bad_lines='skip')

# Kullanılabilir verileri filtrele (resmi olanlar)
df['image_path'] = df.apply(lambda row: os.path.join(IMAGES_DIR, str(row['id']) + '.jpg'), axis=1)
df = df[df.apply(lambda row: os.path.exists(row['image_path']), axis=1)].copy()

# Sınıf etiketlerini sayısal değerlere dönüştürme
df['masterCategory_idx'] = df['masterCategory'].astype('category').cat.codes
class_to_idx = dict(df[['masterCategory', 'masterCategory_idx']].drop_duplicates().sort_values('masterCategory_idx').values)
idx_to_class = {v: k for k, v in class_to_idx.items()}
NUM_CLASSES = len(class_to_idx)

print(f"Toplam {len(df)} adet kullanılabilir resim bulundu.")
print(f"Toplam {NUM_CLASSES} adet ana kategori var: {list(class_to_idx.keys())}")

Toplam 44419 adet kullanılabilir resim bulundu.
Toplam 7 adet ana kategori var: ['Accessories', 'Apparel', 'Footwear', 'Free Items', 'Home', 'Personal Care', 'Sporting Goods']


In [4]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [5]:
class FashionDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
        self.labels = self.df['masterCategory_idx'].values

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.iloc[idx]['image_path']
        label = self.labels[idx]
        
        try:
            image = Image.open(img_path).convert('RGB')
            if self.transform:
                image = self.transform(image)
            return image, label
        except FileNotFoundError:
            print(f"Uyarı: Resim bulunamadı - {img_path}")
            # Hata durumunda geçici bir tensör ve -1 etiketi döndür
            return torch.randn(3, 224, 224), -1

In [7]:
# Veri setini train ve val olarak ayırma
train_size = int(0.8 * len(df))
val_size = len(df) - train_size
# Ayırmanın her zaman aynı sonucu vermesi için bir 'generator' eklemek iyi bir pratiktir
generator = torch.Generator().manual_seed(42)
train_df_subset, val_df_subset = random_split(df, [train_size, val_size], generator=generator)

# Hata almamak için subset'lerden dataframe'e geçerken index'leri sıfırlayalım
train_dataset = FashionDataset(train_df_subset.dataset.iloc[train_df_subset.indices].reset_index(drop=True), transform=data_transforms['train'])
val_dataset = FashionDataset(val_df_subset.dataset.iloc[val_df_subset.indices].reset_index(drop=True), transform=data_transforms['val'])

# --- Sınıf Dengesizliği için Ağırlıklı Örnekleyici (Weighted Sampler) ---
print("\nSınıf dengesizliğini gidermek için sampler oluşturuluyor...")

# 1. Eğitim setindeki her bir sınıfın sayısını hesapla.
#    minlength=NUM_CLASSES, dizinin boyutunun her zaman toplam sınıf sayısı kadar olmasını garanti eder.
class_counts = np.bincount(train_dataset.labels, minlength=NUM_CLASSES)

# 2. Her bir sınıf için ağırlığı hesapla (1 / sayı).
#    Sıfıra bölme hatasını önlemek için sayılara çok küçük bir değer (epsilon) ekliyoruz.
weights = 1.0 / (class_counts.astype(float) + 1e-6)

# 3. Eğitim setindeki her bir örnek için, etiketine karşılık gelen ağırlığı ata.
sample_weights = weights[train_dataset.labels]

# 4. Ağırlıkları PyTorch tensörüne çevir.
sample_weights_tensor = torch.from_numpy(sample_weights).double()

# 5. Sampler'ı oluştur.
sampler = torch.utils.data.WeightedRandomSampler(weights=sample_weights_tensor, num_samples=len(sample_weights_tensor), replacement=True)

# DataLoader'ları oluşturma
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=sampler, num_workers=NUM_WORKERS)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

print("Veri yükleyiciler hazır.")


Sınıf dengesizliğini gidermek için sampler oluşturuluyor...
Veri yükleyiciler hazır.


In [8]:
# Önceden eğitilmiş MobileNetV2 modelini yüklüyoruz
model = torchvision.models.mobilenet_v2(weights='IMAGENET1K_V2')

# Modelin son katmanını (classifier) kendi problemimize (NUM_CLASSES) göre değiştiriyoruz
model.classifier[1] = nn.Linear(model.last_channel, NUM_CLASSES)

# Modeli seçilen cihaza (GPU/CPU) gönderiyoruz
model = model.to(DEVICE)

# Optimizer ve Loss fonksiyonunu tanımlıyoruz
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()

In [ ]:
best_val_accuracy = 0.0
best_model_path = "models/fashion_sense_best_model.pth"
os.makedirs("models", exist_ok=True)

for epoch in range(NUM_EPOCHS):
    # --- Eğitim Aşaması ---
    model.train()
    running_loss = 0.0
    train_corrects = 0
    
    # tqdm ile ilerleme çubuğu ekliyoruz
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Eğitim]"):
        # Bozuk veriyi atla
        if -1 in labels: continue
        
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
        train_corrects += torch.sum(preds == labels.data)
        
    train_loss = running_loss / len(train_loader.dataset)
    train_acc = train_corrects.double() / len(train_loader.sampler) # sampler uzunluğu kullanılır

    # --- Validasyon Aşaması ---
    model.eval()
    val_loss = 0.0
    val_corrects = 0
    
    with torch.no_grad():
        for inputs, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Validasyon]"):
            if -1 in labels: continue
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)
            val_corrects += torch.sum(preds == labels.data)
            
    val_loss = val_loss / len(val_dataset)
    val_acc = val_corrects.double() / len(val_dataset)
    
    print(f"Epoch {epoch+1}/{NUM_EPOCHS} -> Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

    # En iyi modeli kaydetme
    if val_acc > best_val_accuracy:
        best_val_accuracy = val_acc
        torch.save(model.state_dict(), best_model_path)
        print(f"Yeni en iyi model kaydedildi: {best_model_path} (Doğruluk: {val_acc:.4f})")

Epoch 1/10 [Eğitim]:   0%|                                                                                                                                                                                                                                 | 0/556 [00:00<?, ?it/s]

In [ ]:
# En iyi modelin ağırlıklarını yükle
model.load_state_dict(torch.load(best_model_path))
model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in tqdm(val_loader, desc="Son Değerlendirme"):
        if -1 in labels: continue
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Karmaşıklık Matrisini Çizdirme
cm = confusion_matrix(all_labels, all_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=list(class_to_idx.keys()))

fig, ax = plt.subplots(figsize=(10, 10))
disp.plot(ax=ax, xticks_rotation='vertical', cmap='Blues')
plt.title('Karmaşıklık Matrisi (En İyi Model)')
plt.show()